In [12]:
import xarray as xr
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

%run 0A.data_check_n_clean.ipynb

safely removed countries w.o fao/sif/mask data: True 

List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Micronesia (Federated States of)
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Kiribati
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Cook Islands
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Qatar
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Tuvalu
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Marshall Islands
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Tonga
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Nauru
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/French Polynesia
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Niue
List is empty: /Users/caropark/Dropbox (Harvard University)/DataByCountry/Samoa
List i

In [8]:
def grab_vars(start):
    key = pd.read_pickle("./data/calendar_fao_cropkey.pkl")
    idx = np.unique(key['harv_area_idx'].values)

    ## harvested area
    end_harvarea = "/harvest_area_fraction_data_sage_all.nc"
    harvarea = xr.open_dataset(start+end_harvarea)['harvest_area_frac_nearest']
    harvarea['location']= harvarea['location']+0
    harvarea['crop'] = harvarea['crop']+0
    harvarea = harvarea.sel(crop=idx).where(harvarea>0)
    counts = harvarea.groupby('crop').count(...)
    counts = counts.where(counts>0, drop=True)['crop'].values
    if counts.size==0: 
        return
    
    wanted = key[key['harv_area_idx'].isin(counts)]['crop'].values
    harvarea = harvarea.rename({"crop": "harv_area_idx"})
    harvarea = harvarea.where(harvarea.harv_area_idx.isin(counts), drop=True)
    total_harvarea = harvarea.groupby('harv_area_idx').sum('location')
    total_harvarea = total_harvarea.to_dataframe().reset_index().rename(columns={"harvest_area_frac_nearest": "total_harvarea"})
    avg_farmsize = harvarea.where(harvarea.harv_area_idx.isin(counts), drop=True).groupby('harv_area_idx').mean('location')
    avg_farmsize = avg_farmsize.to_dataframe().reset_index().rename(columns={"harvest_area_frac_nearest": "avg_farmsize"})
    areas = total_harvarea.merge(avg_farmsize)

    ### modis cropland types
    end_landtype = "/cropland_fraction_data_modis.nc"
    cropland_frac = xr.open_dataset(start+end_landtype)['cropland_fraction'].mean('year')
    cfrac = pd.DataFrame(xr.merge([cropland_frac, harvarea]).to_dataframe().reset_index().dropna().groupby('harv_area_idx').mean()['cropland_fraction']).reset_index()

    ### majority crop for each country, cropland frac only
    locindex = xr.open_dataset(start+end_landtype)[['lat', 'lon', "location"]].to_pandas().reset_index()
    modis_coords = xr.open_dataset(start+end_landtype)[['lat', 'lon']].set_index(heya=["lat", "lon"]).unstack("heya")
    maj = xr.open_dataset("./data/m2018_majority.nc")["Band1"]
    majcrop = maj.reindex_like(modis_coords, method="nearest")
    majcropdf = majcrop.to_dataframe().reset_index()
    majcropdf = majcropdf.merge(locindex)
    majxr = majcropdf.set_index("location").drop(columns=['lat', 'lon']).to_xarray()
    df= pd.DataFrame(xr.merge([harvarea, majxr]).to_dataframe().reset_index().dropna()[['harv_area_idx', 'Band1']].groupby("harv_area_idx").Band1.value_counts()).rename(columns={"Band1": "counts"}).reset_index()
    majdf = df.loc[df.groupby("harv_area_idx", group_keys=False)['counts'].nlargest(2).index]
    majdf['majcrop_rank']= majdf.groupby('harv_area_idx')['counts'].transform(lambda x: x == x.max()).astype(int).replace(0, 2)
    majdf = majdf.rename(columns={"Band1": "majcrop"}).drop(columns="counts")

    ### putting it all together
    alltog = areas.merge(cfrac).merge(majdf).merge(key[["crop", 'harv_area_idx']], how="left").drop(columns=["harv_area_idx"])
    alltog['country']= start.rsplit('/',1)[1]
    return(alltog)

In [9]:
combined= pd.DataFrame()

for i in allcountries:
    print(i)
    %time
    alltog = grab_vars(i)
    combined = combined.append(alltog)
    print("\n")
    
combined = combined.reset_index(drop=True)

/Users/caropark/Dropbox (Harvard University)/DataByCountry/Jamaica
CPU times: user 4 µs, sys: 7 µs, total: 11 µs
Wall time: 11.9 µs


/Users/caropark/Dropbox (Harvard University)/DataByCountry/Vanuatu
CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 2.86 µs


/Users/caropark/Dropbox (Harvard University)/DataByCountry/Saint Vincent and the Grenadines
CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


/Users/caropark/Dropbox (Harvard University)/DataByCountry/Nepal
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs


/Users/caropark/Dropbox (Harvard University)/DataByCountry/Montenegro
CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 1.91 µs


/Users/caropark/Dropbox (Harvard University)/DataByCountry/Kuwait
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 2.86 µs


/Users/caropark/Dropbox (Harvard University)/DataByCountry/Niger
CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.86 µs


/Users/caropark/Dropbox (Harva

In [10]:
key = pd.read_pickle("./data/calendar_fao_cropkey.pkl")[["crop", "cropname"]]
combined = combined.merge(key)
combined.to_feather("./data/modis_variables.feather")
